<a href="https://colab.research.google.com/github/OMWANDHOJR/Kaggle-Competition/blob/main/Titanic_25_march.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train_df = pd.read_csv('/kaggle/train.csv')
test_df = pd.read_csv('/kaggle/test.csv')


In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_indices, test_indices in split.split(train_df, train_df[['Survived','Pclass','Sex']]):
  strat_train_set = train_df.loc[train_indices]
  strat_test_set = train_df.loc[test_indices]

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


class AgeImputer(BaseEstimator, TransformerMixin):

  def fit(self, X, y=None):
    return self

  def transform(self, X):
    imputer = SimpleImputer(strategy='median')
    X['Age'] = imputer.fit_transform(X[['Age']])
    return X

class FeatureEncoder(BaseEstimator, TransformerMixin):

  def fit(self, X, y=None):
    return self

  def transform(self, X):
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # handle_unknown='ignore' will ignore values not seen during training to prevent errors in the testing phase.
    matrix = encoder.fit_transform(X[['Embarked']])

    # Get feature names from the encoder
    column_names = encoder.get_feature_names_out(['Embarked'])

    # Add the encoded columns to the DataFrame
    for i, col_name in enumerate(column_names):
        X[col_name] = matrix[:, i]

    # Encode 'Sex' feature
    matrix = encoder.fit_transform(X[['Sex']])
    column_names = encoder.get_feature_names_out(['Sex'])  # Get feature names

    # Add encoded columns to the DataFrame
    for i, col_name in enumerate(column_names):
        X[col_name] = matrix[:, i]

    return X

class FeatureDropper(BaseEstimator, TransformerMixin):

  def fit(self, X, y=None):
    return self

  def transform(self, X):
    X = X.drop(['Embarked', 'Name', 'Ticket', 'Cabin','Sex', 'N'], axis=1, errors='ignore')
    return X

In [6]:
pipeline = Pipeline([
  ('age_imputer', AgeImputer()),
  ('feature_encoder', FeatureEncoder()),
  ('feature_dropper', FeatureDropper())
])

In [7]:
strat_train_set = pipeline.fit_transform(strat_train_set)
strat_test_set = pipeline.fit_transform(strat_test_set)




In [13]:
test_df = pipeline.fit_transform(test_df)

In [8]:
strat_train_set.drop('Embarked_nan', axis=1)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
867,868,0,1,31.0,0,0,50.4958,0.0,0.0,1.0,0.0,1.0
619,620,0,2,26.0,0,0,10.5000,0.0,0.0,1.0,0.0,1.0
50,51,0,3,7.0,4,1,39.6875,0.0,0.0,1.0,0.0,1.0
286,287,1,3,30.0,0,0,9.5000,0.0,0.0,1.0,0.0,1.0
834,835,0,3,18.0,0,0,8.3000,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
482,483,0,3,50.0,0,0,8.0500,0.0,0.0,1.0,0.0,1.0
518,519,1,2,36.0,1,0,26.0000,0.0,0.0,1.0,1.0,0.0
54,55,0,1,65.0,0,1,61.9792,1.0,0.0,0.0,0.0,1.0
603,604,0,3,44.0,0,0,8.0500,0.0,0.0,1.0,0.0,1.0


In [9]:
strat_test_set.drop('Survived', axis=1)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
659,660,1,58.00,0,2,113.2750,1.0,0.0,0.0,0.0,1.0
119,120,3,2.00,4,2,31.2750,0.0,0.0,1.0,1.0,0.0
875,876,3,15.00,0,0,7.2250,1.0,0.0,0.0,1.0,0.0
67,68,3,19.00,0,0,8.1583,0.0,0.0,1.0,0.0,1.0
644,645,3,0.75,2,1,19.2583,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
85,86,3,33.00,3,0,15.8500,0.0,0.0,1.0,1.0,0.0
687,688,3,19.00,0,0,10.1708,0.0,0.0,1.0,0.0,1.0
538,539,3,29.00,0,0,14.5000,0.0,0.0,1.0,0.0,1.0
279,280,3,35.00,1,1,20.2500,0.0,0.0,1.0,1.0,0.0


In [10]:

from sklearn.preprocessing import StandardScaler

X = strat_train_set.drop('Survived', axis=1)
y = strat_train_set['Survived']

scaler = StandardScaler()
X_data = scaler.fit_transform(X)
y_data = y.to_numpy()



In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler # Import StandardScaler


# Assuming 'X_data' and 'y_data' are your features and target variable
# obtained after the pipeline transformation

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# Define the models
model1 = LogisticRegression()  # Logistic Regression
model2 = RandomForestClassifier()  # Random Forest
model3 = SVC()  # Support Vector Machine
model4 = KNeighborsClassifier()  # K-Nearest Neighbors


# Create a pipeline with scaling for each model
pipeline_model1 = Pipeline([('scaler', StandardScaler()), ('model', model1)])
pipeline_model2 = Pipeline([('scaler', StandardScaler()), ('model', model2)])
pipeline_model3 = Pipeline([('scaler', StandardScaler()), ('model', model3)])
pipeline_model4 = Pipeline([('scaler', StandardScaler()), ('model', model4)])

# Train the models using the training data
pipeline_model1.fit(X_train, y_train)
pipeline_model2.fit(X_train, y_train)
pipeline_model3.fit(X_train, y_train)
pipeline_model4.fit(X_train, y_train)


# Make predictions on the testing data
y_pred1 = pipeline_model1.predict(X_test)
y_pred2 = pipeline_model2.predict(X_test)
y_pred3 = pipeline_model3.predict(X_test)
y_pred4 = pipeline_model4.predict(X_test)

# Evaluate the accuracy of each model
accuracy1 = accuracy_score(y_test, y_pred1)
accuracy2 = accuracy_score(y_test, y_pred2)
accuracy3 = accuracy_score(y_test, y_pred3)
accuracy4 = accuracy_score(y_test, y_pred4)

# Print the accuracy of each model
print(f"Accuracy of Logistic Regression (model1): {accuracy1}")
print(f"Accuracy of Random Forest (model2): {accuracy2}")
print(f"Accuracy of SVM (model3): {accuracy3}")
print(f"Accuracy of KNN (model4): {accuracy4}")

Accuracy of Logistic Regression (model1): 0.7902097902097902
Accuracy of Random Forest (model2): 0.8321678321678322
Accuracy of SVM (model3): 0.7832167832167832
Accuracy of KNN (model4): 0.8041958041958042


In [14]:
predictions = pipeline_model4.predict(test_df.to_numpy())

ValueError: X has 11 features, but StandardScaler is expecting 12 features as input.

In [18]:
print("Train columns:", strat_test_set.columns)
print("Test columns:", test_df.columns)


Train columns: Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Sex_female', 'Sex_male'],
      dtype='object')
Test columns: Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C',
       'Embarked_Q', 'Embarked_S', 'Sex_female', 'Sex_male'],
      dtype='object')


In [ ]:
predictions

array([1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 0])

In [ ]:
final_df = pd.DataFrame({'PassengerId': strat_test_set['PassengerId'], 'Survived': predictions})

final_df.to_csv('/kaggle/submission.csv', index=False)